In [ ]:
from monai.networks.nets import UNet

model = UNet(3, 1, 1, 
                      channels=[],
                      strides=[],
                      kernel_size=3,
                      up_kernel_size=3,
                      dropout=0.4
                        )

In [39]:
from monai.transforms import (
    AsDiscrete,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    SaveImaged,
    ScaleIntensityRanged,
    Spacingd,
    Invertd,
)

train_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        ScaleIntensityRanged(
            keys=["image"],
            a_min=-57,
            a_max=164,
            b_min=0.0,
            b_max=1.0,
            clip=True,
        ),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        Spacingd(keys=["image", "label"], pixdim=(1.5, 1.5, 2.0), mode=("bilinear", "nearest")),
        RandCropByPosNegLabeld(
            keys=["image", "label"],
            label_key="label",
            spatial_size=(96, 96, 96),
            pos=1,
            neg=1,
            num_samples=4,
            image_key="image",
            image_threshold=0,
        ),
        # user can also add other random transforms
        # RandAffined(
        #     keys=['image', 'label'],
        #     mode=('bilinear', 'nearest'),
        #     prob=1.0, spatial_size=(96, 96, 96),
        #     rotate_range=(0, 0, np.pi/15),
        #     scale_range=(0.1, 0.1, 0.1)),
    ]
)


val_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        ScaleIntensityRanged(
            keys=["image"],
            a_min=-57,
            a_max=164,
            b_min=0.0,
            b_max=1.0,
            clip=True,
        ),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        Spacingd(keys=["image", "label"], pixdim=(1.5, 1.5, 2.0), mode=("bilinear", "nearest")),
    ]
)

monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.


In [40]:
import glob
import os

data_dir = '..\\output_dir'

train_images = sorted(glob.glob(os.path.join(data_dir, "images", "*.nii.gz")))
train_masks = sorted(glob.glob(os.path.join(data_dir, "masks", "*.nii.gz")))

print(train_images)
print(train_masks)

data_dicts = [{
            "image": image_name,
            "mask": mask_name} for image_name, mask_name in zip(train_images, train_masks)]

['..\\output_dir\\images\\image_0.nii.gz']
['..\\output_dir\\masks\\mask0Prostate.nii.gz']


In [ ]:
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.utils import first, set_determinism
import matplotlib.pyplot as plt

set_determinism(seed=0)
# train_files, val_files = data_dicts[:-13], data_dicts[-13:]
val_files = data_dicts[0]
print(val_files)
check_ds = Dataset(data=val_files, transform=val_transforms)
check_loader = DataLoader(check_ds, batch_size=1)

# Access data using DataLoader
for batch_data in check_loader:
    images = batch_data["image"]
    labels = batch_data["mask"]
    print(images.shape, labels.shape)  # Print the shapes of the images and labels
    break  # exit after one batch

image, label = (check_data["image"][0][0], check_data["mask"][0][0])

print(f"image shape: {image.shape}, label shape: {label.shape}")

# plot the slice [:, :, 80]
plt.figure("check", (12, 6))
plt.subplot(1, 2, 1)
plt.title("image")
plt.imshow(image[:, :, 80], cmap="gray")
plt.subplot(1, 2, 2)
plt.title("label")
plt.imshow(label[:, :, 80])
plt.show()

{'image': '..\\output_dir\\images\\image_0.nii.gz', 'mask': '..\\output_dir\\masks\\mask0Prostate.nii.gz'}


KeyError: 0